In [1]:
%load_ext autoreload
%load_ext autotime

time: 191 µs (started: 2024-10-09 15:43:39 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 538 µs (started: 2024-10-09 15:43:39 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 753 µs (started: 2024-10-09 15:43:39 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-cm1-node-32"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/cm1-node-32"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-32/v1/COMPACT/*.pfw.gz"


time: 638 µs (started: 2024-10-09 15:43:39 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 5.73 s (started: 2024-10-09 15:43:39 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 1min 25s (started: 2024-10-09 15:43:45 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          280.35 seconds                                                                                │
│  I/O Time         2.25 seconds                                                                                  │
│                   ├── Read - 0.03 seconds (1.48%)                                                               │
│                   ├── Write - 0.08 seconds (3.56%)                                                              │
│                   └── Metadata - 2.15 seconds (95.74%)                                                          │
│  I/O Operations   2,587,987 ops                                                                                 │
│                   ├── Read - 256,001 ops (9.89%)                                                                │
│                   ├── Write - 245,617 ops (9.49%)                                                               │
│                   └── Metadata - 2,077,153 ops (80.26%)                                                         │
│  I/O Size         4.64 GiB                                                                                      │
│                   ├── Read - 27.84 MiB (0.59%)                                                                  │
│                   └── Write - 4.61 GiB (99.41%)                                                                 │
│  Read Requests    4 kiB-16 kiB - 256,001 ops                                                                    │
│                   ├── <4 kiB - 254,976 ops (99.60%)                                                             │
│                   └── 4-16 kiB - 1,025 ops (0.40%)                                                              │
│  Write Requests   4 kiB-64 kiB - 245,617 ops                                                                    │
│                   ├── <4 kiB - 133,273 ops (54.26%)                                                             │
│                   ├── 4-16 kiB - 43,736 ops (17.81%)                                                            │
│                   └── 16-64 kiB - 68,608 ops (27.93%)                                                           │
│  Nodes            32 nodes                                                                                      │
│                   ├── 13788 - 2.25 s (100.00%) - 887.05 kiB/144.52 MiB R/W (0.02/3.04%) - 80,206 ops (3.10%)    │
│                   ├── 31683 - 1.89 s (84.24%) - 897.63 kiB/150.23 MiB R/W (0.02/3.16%) - 83,211 ops (3.22%)     │
│                   ├── 7375 - 1.40 s (62.31%) - 891.13 kiB/148.62 MiB R/W (0.02/3.13%) - 81,052 ops (3.13%)      │
│                   ├── 27953 - 1.40 s (62.21%) - 893.22 kiB/148.62 MiB R/W (0.02/3.13%) - 81,007 ops (3.13%)     │
│                   ├── 475 - 1.40 s (62.12%) - 891.22 kiB/144.52 MiB R/W (0.02/3.04%) - 80,023 ops (3.09%)       │
│                   ├── 46749 - 1.39 s (61.76%) - 888.74 kiB/148.62 MiB R/W (0.02/3.13%) - 81,141 ops (3.14%)     │
│                   ├── 7923 - 1.39 s (61.60%) - 891.05 kiB/148.62 MiB R/W (0.02/3.13%) - 81,314 ops (3.14%)      │
│                   ├── 42785 - 1.38 s (61.52%) - 890.97 kiB/148.62 MiB R/W (0.02/3.13%) - 81,081 ops (3.13%)     │
│                   ├── 36834 - 1.38 s (61.32%) - 891.84 kiB/148.62 MiB R/W (0.02/3.13%) - 81,068 ops (3.13%)     │
│                   ├── 54829 - 1.38 s (61.22%) - 891.56 kiB/144.52 MiB R/W (0.02/3.04%) - 80,125 ops (3.10%)     │
│                   ├── 19921 - 1.38 s (61.14%) - 889.03 kiB/148.62 MiB R/W (0.02/3.13%) - 80,948 ops (3.13%)     │
│                   ├── 27579 - 1.37 s (60.97%) - 891.11 kiB/148.62 MiB R/W (0.02/3.13%) - 81,062 ops (3.13%)     │
│                   ├── 13118 - 1.37 s (60.93%) - 890.48

time: 5.11 s (started: 2024-10-09 15:45:11 -07:00)
